In [15]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Load the Excel file
try:
    df = pd.read_excel('Sales_CG_DL_17_25.xlsx')
except FileNotFoundError:
    print("Error: 'sales_dataset.xlsx' not found. Please make sure the file is in the correct directory.")
    exit()

# Convert 'Billing Date' to datetime objects
df['Billing Date'] = pd.to_datetime(df['Billing Date'])

# --- User defined variables for filtering ---
selected_district = 'BILASPUR' # Replace with actual district
selected_mrp = 5 # Replace with actual MRP value
# ------------------------------------------

# Filter data first
df_filtered = df[(df['District'] == selected_district) & (df['MRP'] == selected_mrp)].copy()

# IMPORTANT: Aggregate 'Billed Quantity' by 'Billing Date' to handle duplicates
# This sums the quantities for each unique date.
df_daily_sales = df_filtered.groupby('Billing Date')['Billed Quantity'].sum().reset_index()

# Set 'Billing Date' as index and sort
df_daily_sales.set_index('Billing Date', inplace=True)
df_daily_sales.sort_index(inplace=True)

# Create a full date range from min to max date in your *aggregated* data
full_date_range = pd.date_range(start=df_daily_sales.index.min(), end=df_daily_sales.index.max(), freq='D')

# Reindex the dataframe to include all dates and fill missing 'Billed Quantity' with 0
# Now df_daily_sales has a unique index, so reindex should work.
df_resampled = df_daily_sales.reindex(full_date_range, fill_value=0)

# If you only want the 'Billed Quantity' for the model:
sales_data = df_resampled['Billed Quantity'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
sales_data_normalized = scaler.fit_transform(sales_data)

# Create sequences for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

sequence_length = 30 # You can tune this hyperparameter (e.g., 30 days)
X, y = create_sequences(sales_data_normalized, sequence_length)

# Convert to PyTorch tensors
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

# Split data into training and testing (e.g., everything up to 2024 for training)
# Find the index for the start of 2024
train_end_date = pd.to_datetime('2023-12-31')

# Ensure train_end_date is within the resampled DataFrame's index range
if train_end_date not in df_resampled.index:
    # If the exact date isn't there, find the nearest preceding date
    train_idx_end = df_resampled.index.get_loc(train_end_date, method='ffill')
else:
    train_idx_end = df_resampled.index.get_loc(train_end_date)


# Adjust train_idx_end to account for sequence_length
# The last sequence for training should end at train_end_date
num_sequences = len(df_resampled) - sequence_length
# This is the point in the X and y arrays where the 2024 data (test data) begins.
train_split_point = train_idx_end - sequence_length + 1

# Check if train_split_point is valid
if train_split_point < 0:
    print("Warning: Not enough data to create sequences before 2024 with the chosen sequence length.")
    print("Adjusting train_split_point to 0. You might need more historical data or a smaller sequence_length.")
    train_split_point = 0
elif train_split_point > len(X):
    print("Error: train_split_point is beyond the available sequences. Check your date ranges and sequence length.")
    exit()


X_train = X[:train_split_point]
y_train = y[:train_split_point]

X_test = X[train_split_point:]
y_test = y[train_split_point:] # These are the actual values for 2024

# Move to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

Using device: cuda


In [16]:
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1) # Shape: (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, d_model)
        # We need to add positional encoding to the last dimension of x, for each position.
        # self.pe is (max_len, 1, d_model)
        # x is (batch_size, sequence_length, d_model)
        # So, we need to select relevant positional encodings based on sequence length.
        return x + self.pe[:x.size(1), :].transpose(0, 1) # Transpose to (1, seq_len, d_model) then add


class SalesTransformer(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Linear(input_size, d_model) # Project input_size to d_model
        self.positional_encoding = PositionalEncoding(d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)

        self.fc_out = nn.Linear(d_model, output_size) # Output layer

    def forward(self, src):
        # src shape: (batch_size, sequence_length, input_size)
        src = self.embedding(src) * math.sqrt(self.d_model) # Apply embedding and scale
        src = self.positional_encoding(src) # Add positional encoding

        # The transformer encoder expects (batch_size, sequence_length, d_model) with batch_first=True
        output = self.transformer_encoder(src)

        # We take the output of the last time step for prediction, similar to LSTM
        # Or, you can average/pool outputs if predicting a sequence summary
        predictions = self.fc_out(output[:, -1, :]) # Taking the last sequence element's output
        return predictions

# Model parameters
input_size = 1 # Billed Quantity
d_model = 64 # Dimension of the embedding and transformer layers
nhead = 4 # Number of attention heads
num_encoder_layers = 2 # Number of transformer encoder layers
dim_feedforward = 128 # Dimension of the feedforward network model in nn.TransformerEncoderLayer
output_size = 1
dropout = 0.1

model = SalesTransformer(input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size, dropout)
model.to(device)

SalesTransformer(
  (embedding): Linear(in_features=1, out_features=64, bias=True)
  (positional_encoding): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
)

In [17]:
import torch.optim as optim

# Loss and Optimizer
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Learning rate can be tuned

epochs = 100 # Number of training epochs, can be tuned

for i in range(epochs):
    model.train() # Set model to training mode
    optimizer.zero_grad() # Clear gradients

    y_pred = model(X_train)
    single_loss = loss_function(y_pred, y_train)

    single_loss.backward() # Backpropagation
    optimizer.step() # Update weights

    if i % 10 == 0:
        print(f'Epoch {i} Loss: {single_loss.item()}')

print("Training complete.")

Epoch 0 Loss: 3.6857268810272217
Epoch 10 Loss: 0.15676280856132507
Epoch 20 Loss: 0.11246347427368164
Epoch 30 Loss: 0.09554146230220795
Epoch 40 Loss: 0.07648752629756927
Epoch 50 Loss: 0.0690046176314354
Epoch 60 Loss: 0.06615505367517471
Epoch 70 Loss: 0.06607531011104584
Epoch 80 Loss: 0.06650837510824203
Epoch 90 Loss: 0.06579100340604782
Training complete.


In [18]:
model.eval() # Set model to evaluation mode
predicted_sales_2024_2025_normalized = []

with torch.no_grad():
    # Start with the last sequence from the training data to make the first prediction for 2024-01-01
    current_prediction_sequence = X_train[-1:].to(device)

    # Determine the range of dates to predict (2024-01-01 to 2025-04-30)
    prediction_start_date = pd.to_datetime('2024-01-01')
    prediction_end_date = pd.to_datetime('2025-04-30') # Predict up to April 2025
    prediction_dates_range = pd.date_range(start=prediction_start_date, end=prediction_end_date, freq='D')
    num_prediction_steps = len(prediction_dates_range)

    for i in range(num_prediction_steps):
        prediction = model(current_prediction_sequence)
        predicted_sales_2024_2025_normalized.append(prediction.item())

        # Update the sequence for the next prediction
        # Remove the oldest value and add the new prediction
        next_input_value = prediction.view(1, 1, 1) # Reshape for concatenation (batch, seq_len, features)
        current_prediction_sequence = torch.cat((current_prediction_sequence[:, 1:, :], next_input_value), dim=1)


# Inverse transform the predictions
predicted_sales_2024_2025 = scaler.inverse_transform(np.array(predicted_sales_2024_2025_normalized).reshape(-1, 1)).flatten()

# Get the actual 2024 sales data for comparison (up to 2024-12-31)
actual_2024_data_resampled = df_resampled[(df_resampled.index >= pd.to_datetime('2024-01-01')) & (df_resampled.index <= pd.to_datetime('2024-12-31'))]
actual_sales_2024 = actual_2024_data_resampled['Billed Quantity'].values

# For plotting, we only need 2024 predictions and actuals
predicted_sales_2024_for_plot = predicted_sales_2024_2025[:len(actual_sales_2024)]

# Ensure both arrays have the same length for plotting
min_len_plot = min(len(predicted_sales_2024_for_plot), len(actual_sales_2024))
predicted_sales_2024_for_plot = predicted_sales_2024_for_plot[:min_len_plot]
actual_sales_2024 = actual_sales_2024[:min_len_plot]

In [19]:
# Create a DataFrame for 2024 actual dates
dates_2024_for_plot = pd.date_range(start='2024-01-01', periods=len(predicted_sales_2024_for_plot), freq='D')
predicted_df_2024 = pd.DataFrame({'Date': dates_2024_for_plot, 'Predicted_Quantity': predicted_sales_2024_for_plot})
actual_df_2024 = pd.DataFrame({'Date': dates_2024_for_plot, 'Actual_Quantity': actual_sales_2024})

# Resample to monthly sums for 2024
predicted_monthly_2024 = predicted_df_2024.set_index('Date').resample('M')['Predicted_Quantity'].sum()
actual_monthly_2024 = actual_df_2024.set_index('Date').resample('M')['Actual_Quantity'].sum()

# Create a DataFrame for plotting 2024
plot_df_2024 = pd.DataFrame({
    'Month': predicted_monthly_2024.index.strftime('%b'),
    'Predicted Sales': predicted_monthly_2024.values,
    'Actual Sales': actual_monthly_2024.values
})

# Plotting 2024 sales
plt.figure(figsize=(12, 6))
plt.plot(plot_df_2024['Month'], plot_df_2024['Predicted Sales'], label='Predicted 2024 Sales', marker='o')
plt.plot(plot_df_2024['Month'], plot_df_2024['Actual Sales'], label='Actual 2024 Sales', marker='x')
plt.title(f'2024 Monthly Sales Prediction vs Actuals for District: {selected_district}, MRP: {selected_mrp}')
plt.xlabel('Month')
plt.ylabel('Billed Quantity')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# --- Plotting full prediction range up to April 2025 (Optional but good for future planning) ---
predicted_full_df = pd.DataFrame({'Date': prediction_dates_range, 'Predicted_Quantity': predicted_sales_2024_2025})
predicted_full_monthly = predicted_full_df.set_index('Date').resample('M')['Predicted_Quantity'].sum()

plt.figure(figsize=(15, 7))
plt.plot(predicted_full_monthly.index, predicted_full_monthly.values, label='Predicted Sales (2024-2025)', marker='o')
plt.title(f'Monthly Sales Prediction (2024-April 2025) for District: {selected_district}, MRP: {selected_mrp}')
plt.xlabel('Date')
plt.ylabel('Billed Quantity')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Optional: You can also combine actuals and predictions for a single plot if you have actuals beyond 2024
# For example, if you want to plot up to April 2025, you'd need actuals for those months from df_resampled
# and then combine. For now, focusing on 2024 actuals vs predictions.